In [1]:
import twitterGraph as tg
import numpy as np
import random as rand
import tweepy
from config import tw_key, tw_secret
graph = tg.twitterGraph()
auth = auth = tweepy.AppAuthHandler(tw_key, tw_secret)
api = tweepy.API(auth)

In [35]:
l = api.lists_all("CSPAN")

In [48]:
tg.twitterID

dtype([('id', '<u4'), ('followers', '<u4'), ('followersPosition', '<u8'), ('friends', '<u4'), ('friendsPosition', '<u8')])

In [37]:
congress = l[16]

In [45]:
graph.getRandomUser()

(57928466, 5, 6096517408, 20, 5889269600)

In [23]:
ids = [x.id for x in congress.members()]

In [51]:
graph.getRandomUser()

(42416832, 14, 5153755860, 10, 3827417728)

In [14]:
graph.getUser(12)

(12, 1001061, 0, 586, 0)

In [47]:
[x in templist for x in graph.getFollowers(temp)]

[False, False, False, False, True, False]

In [17]:
type(12475837589256356)

int